# JurisCape Node D: The Judge (DeepSeek-R1)
**Role**: Legal Reasoning and Verdict Generation.
**Model**: `deepseek-ai/DeepSeek-R1-Distill-Llama-8B` (4-bit via Unsloth).

In [ ]:
# 1. Install Dependencies
!pip install fastapi uvicorn pyngrok python-multipart nest_asyncio requests
!pip install unsloth vllm

In [ ]:
# 2. Load DeepSeek-R1 (Distill 8B)
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)
print("Judge is Ready!")

In [ ]:
# 3. Define Judgment Logic
def get_verdict(facts, legal_sections):
    prompt_template = """
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    You are a strictly logical Judge. Analyze the following FACTS against the LAWS.
    Determine 'Guilty' or 'Not Guilty' and provide STEP-BY-STEP reasoning.

    ### Input:
    FACTS: {facts}
    LAWS: {laws}

    ### Response:
    """
    
    inputs = tokenizer(
        prompt_template.format(facts=facts, laws=legal_sections),
        return_tensors = "pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    return tokenizer.batch_decode(outputs)[0]


In [ ]:
# 4. Start Server
from pyngrok import ngrok
import uvicorn
import os
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List

NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"
ngrok.set_auth_token(NGROK_TOKEN)

SWARM_SECRET = "change-me-in-prod-secure-swarm-key"
app = FastAPI()

@app.middleware("http")
async def verify_secret(request: Request, call_next):
    if request.url.path in ["/docs", "/openapi.json"]:
        return await call_next(request)
    if request.headers.get("X-Swarm-Secret") != SWARM_SECRET:
        return JSONResponse(status_code=403, content={"detail": "Unauthorized"})
    return await call_next(request)

class JudgmentRequest(BaseModel):
    facts: List[str]
    evidence_summary: str
    legal_sections: List[str]

@app.post("/adjudicate")
async def adjudicate(payload: JudgmentRequest):
    print("Deliberating...")
    raw_verdict = get_verdict(payload.facts, payload.legal_sections)
    return {
        "raw_output": raw_verdict,
        "summary": "[Processed] Verdict logic executed."
    }

# CLEANUP & RUN
ngrok.kill()
os.system("pkill ngrok")
tunnel = ngrok.connect(8000)
print(f"\n=== PUBLIC URL: {tunnel.public_url} ===\n")
config = uvicorn.Config(app, port=8000)
await uvicorn.Server(config).serve()